In [1]:
# The workflow for analyzing the routes begins with coordinate points used as origin locations.
import pandas as pd

df = pd.DataFrame({
    "city_name": ["Umeå", "Dresden", "Barcelona"],
    "region": ["europe", "europe", "europe"],
    "country": ["Sweden", "Germany", "Spain"],
})

# To sample nodes from the cities we call use a method from sample_nodes.py
import node_sampling
df = node_sampling.get_random_nodes_for_all_cities(df,min_distance_km=3)
display(df)

Random nodes for Umeå that are at least 4 km apart: [7638448958, 512242312, 182795832]
Random nodes for Umeå: [7638448958, 512242312, 182795832]
Node: 7638448958
Random node coordinates for Umeå: 63.8757082, 20.2706017
Node: 512242312
Random node coordinates for Umeå: 63.8510012, 20.2184277
Node: 182795832
Random node coordinates for Umeå: 63.8528983, 20.3389278
Random nodes for Dresden that are at least 4 km apart: [21540856, 18628040, 1274534961]
Random nodes for Dresden: [21540856, 18628040, 1274534961]
Node: 21540856
Random node coordinates for Dresden: 51.0567385, 13.6873738
Node: 18628040
Random node coordinates for Dresden: 51.0473951, 13.7896455
Node: 1274534961
Random node coordinates for Dresden: 51.0352885, 13.7400738
Random nodes for Barcelona that are at least 4 km apart: [8119767440, 11369675082, 598221293]
Random nodes for Barcelona: [8119767440, 11369675082, 598221293]
Node: 8119767440
Random node coordinates for Barcelona: 41.4028542, 2.1183168
Node: 11369675082
Random

,city_name,country,region,network_type,node1_id,node1_latlon,node2_id,node2_latlon,node3_id,node3_latlon
0,Umeå,Sweden,europe,drive,7638448958,"(63.8757082, 20.2706017)",512242312,"(63.8510012, 20.2184277)",182795832,"(63.8528983, 20.3389278)"
1,Dresden,Germany,europe,drive,21540856,"(51.0567385, 13.6873738)",18628040,"(51.0473951, 13.7896455)",1274534961,"(51.0352885, 13.7400738)"
2,Barcelona,Spain,europe,drive,8119767440,"(41.4028542, 2.1183168)",11369675082,"(41.3758069, 2.1773961)",598221293,"(41.4356064, 2.2009057)"
